In [ ]:
%matplotlib inline
from ui.graph import ModelPieChart
from model.projects import StandardProject
from IPython.core.display_functions import display

p = StandardProject()
g = ModelPieChart(p)
display(g.get_widget())